Imports

In [11]:
import numpy as np

import datetime as dt
import panel as pn

pn.extension()
import panel as pn
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import json 
import pandas as pd 
# from pandas.json_normalize import json_normalize #package for flattening json in pandas df

import pathlib

import holoviews as hv
import matplotlib.pyplot as plt
import hvplot
import param
import hvplot.pandas
from bokeh.models import GlyphRenderer, LinearAxis, LinearScale, Range1d


pn.extension(sizing_mode='stretch_width')

pn.extension('perspective')
from cognite.client.utils import MIN_TIMESTAMP_MS, MAX_TIMESTAMP_MS

### Configure cognite client

In [12]:
# Contact Project Administrator to get these
TENANT_ID = "3aa4a235-b6e2-48d5-9195-7fcf05b459b0" # AAD using @equinor.com
CLIENT_ID = "3d7a85da-05ab-4675-b5f9-0c4b96a1c84b" # CDF_ALL_USER_APPLICATION_FLOW
CDF_CLUSTER = "westeurope-1"
# -------------for DEV project-----------------
COGNITE_PROJECT = "equinor-dev"

# -------------for TEST project-----------------
# COGNITE_PROJECT = "equinor-test"


from cognite.client import CogniteClient
client = CogniteClient.default_oauth_interactive(
    project=COGNITE_PROJECT,
    tenant_id=TENANT_ID,
    client_id=CLIENT_ID,
    cdf_cluster=CDF_CLUSTER,
    client_name="my-cognite-python-test", # a name to identify your session
)
from cognite.client.data_classes import LabelFilter



### Define Dataset

In [13]:
#  
wellcom_dataset = "src:015:wellcom"
witsml_dataset = "src:014:witsmldata"
active_wellbore = "active:wellbore"

#list wellcom wellbores that are active in WITSML
active_wellbores = client.assets.list(data_set_external_ids = wellcom_dataset, labels=LabelFilter(contains_all=[active_wellbore]), limit=-1)
#Get sensor data 
witsml_db  = client.time_series.list(data_set_external_ids = witsml_dataset, limit=-1)

### Convert to pandas dataframe and flatten

In [147]:
# Wellbores
pd_active_wellbores = active_wellbores.to_pandas(metadata_prefix=active_wellbores)

# Sensordata
_pd_witsml = witsml_db.to_pandas(metadata_prefix=witsml_db)
_pd_witsml.rename(columns ={'name': 'sensorName'} , inplace = True)
# Flatten Sensordata
_pd_witsml_flattened = pd.concat([_pd_witsml.drop(['metadata'], axis=1), pd.json_normalize(_pd_witsml['metadata'] )], axis=1)





#Limiting columns
_pd_witsml_flattened_lim = _pd_witsml_flattened[['id', 'uidWell', 'sensorName', 'name', 'uidWellbore',  'nameWellbore','external_id',  'is_string', 'asset_id', 'is_step',
       'description', 'security_categories', 'data_set_id', 'created_time',
       'last_updated_time', 'type', 'commonData_dTimCreation',
       'commonData_dTimLastChange', 'commonData_itemState',
       'commonData_sourceName', 'creationDate', 'direction',
       'endDateTimeIndex', 'indexCurve', 'indexType',  'nameWell',
       'nullValue', 'objectGrowing', 'serviceCompany',
       'startDateTimeIndex',  'uidLog',
       'unavailable_in_source', 'witsml-server-ref', 'source',
       'curveDescription', 'maxDateTimeIndex', 'minDateTimeIndex', 'mnemAlias',
       'mnemonic', 'typeLogData', 'uid']]



_pd_witsml_flattened_lim.head(10)

,id,uidWell,sensorName,name,uidWellbore,nameWellbore,external_id,is_string,asset_id,is_step,...,unavailable_in_source,witsml-server-ref,source,curveDescription,maxDateTimeIndex,minDateTimeIndex,mnemAlias,mnemonic,typeLogData,uid
0,881650803765,49a36410-e7fb-4864-b3c7-c1eba937dcd9,CEMPUMPPR,Surface Logging Data - Time RT,0ac9cfa3-b6a0-4af3-85e4-3180f68f0c11,NO 16/2-D-2 T4,dw:witsml:log.49a36410-e7fb-4864-b3c7-c1eba937...,False,2033175562258369,False,...,false,witsml-server-reference,KongsbergDigital,Cementing Pump Pressure,2023-01-11T17:29:47.000000Z,2022-11-24T05:27:00.000000Z,CEMPUMPPR,CEMPUMPPR,double,CEMPUMPPR
1,1641460701259,4eccc70b-adba-442b-852c-8f1ab7cbcf77,MROUT,Surface Logging Data - Time RT,b467d241-4086-47c5-9c6a-a6fc8cd010ce,NO 16/2-D-10,dw:witsml:log.4eccc70b-adba-442b-852c-8f1ab7cb...,False,8564953405341906,False,...,false,witsml-server-reference,KongsbergDigital,Mud Resistivity Out,2019-08-22T08:40:57.000000Z,2019-08-22T03:56:45.000000Z,MROUT,MROUT,double,MROUT
2,2811296096383,e17da3e3-6e4f-4fdb-a53b-3a43392721ef,CEMWTIN,Surface Logging Data - Time RT,dc0ea5a6-c45c-4e1b-a57b-cde2bdd3af2c,NO 16/2-D-42,dw:witsml:log.e17da3e3-6e4f-4fdb-a53b-3a433927...,False,8568419009254753,False,...,false,witsml-server-reference,KongsbergDigital,Cementing Fluid Density In,2021-04-30T18:29:10.000000Z,2020-06-04T08:59:00.000000Z,CEMWTIN,CEMWTIN,double,CEMWTIN
3,3562630500496,aa2787e0-bd34-40ba-9783-6426a3d0f952,PIT30,Surface Logging Data - Time RT,542e790e-9b76-40a6-ab97-3b75e1ebfa8c,NO 16/2-H-3 AH,dw:witsml:log.aa2787e0-bd34-40ba-9783-6426a3d0...,False,2333496463722180,False,...,false,witsml-server-reference,KongsbergDigital,Pit 30 Volume,2023-07-28T20:23:30.000000Z,2023-07-09T20:56:30.000000Z,PIT30,PIT30,double,PIT30
4,4349328030238,e5912051-eec4-4b18-9dd5-bc687894a406,KILL_PRESS,Surface Logging Data - Time RT,b2598fc0-bcce-4120-a51c-5006ff331fad,NO 16/2-D-37,dw:witsml:log.e5912051-eec4-4b18-9dd5-bc687894...,False,6803620337242231,False,...,false,witsml-server-reference,KongsbergDigital,Kill Pressure,2022-10-12T08:05:02.000000Z,2022-02-06T01:26:18.000000Z,KILL_PRESS,KILL_PRESS,double,KILL_PRESS
5,4429072237890,742cff34-6c07-4a7b-927b-ad2480600aaa,PIT12,Surface Logging Data - Time RT,b6d22545-b216-44d3-ae3b-68d1d0d8037f,NO 16/2-D-4,dw:witsml:log.742cff34-6c07-4a7b-927b-ad248060...,False,5156760718670218,False,...,false,witsml-server-reference,KongsbergDigital,Pit 12 Volume,2022-04-12T05:54:30.000000Z,2022-03-31T14:09:30.000000Z,PIT12,PIT12,double,PIT12
6,4541370327609,bf8e587b-49d0-4195-8207-1e1a6d9e80f5,DACECDFRACP,Surface Logging Data - Time RT,779b7e67-58c3-432b-80e8-9aea86968d55,NO 16/2-H-2 H,dw:witsml:log.bf8e587b-49d0-4195-8207-1e1a6d9e...,False,4409484861468177,False,...,false,witsml-server-reference,KongsbergDigital,Value of corrected simulated ECD at point clos...,2023-06-04T21:09:00.000000Z,2022-11-10T10:44:49.000000Z,DACECDFRACP,DACECDFRACP,double,DACECDFRACP
7,5111276005195,5a07f5ab-b3e0-4bd1-8a5d-8a1a72a75987,RH_RPM6_COR50,Surface Logging Data - Time RT,7c135cc1-5b81-4ef7-be77-adad018e5e17,NO 16/2-D-43,dw:witsml:log.5a07f5ab-b3e0-4bd1-8a5d-8a1a72a7...,False,1908006421875178,False,...,false,witsml-server-reference,KongsbergDigital,Mud engineers 6RPM 50C corrected,2022-11-15T11:35:40.000000Z,2022-01-26T07:42:50.000000Z,RH_RPM6_COR50,RH_RPM6_COR50,double,RH_RPM6_COR50
8,5845006621735,e5912051-eec4-4b18-9dd5-bc687894a406,BLOCKCOMP,Surface Logging Data - Time RT,b2598fc0-bcce-4120-a51c-5006ff331fad,NO 16/2-D-37,dw:witsml:log.e5912051-eec4-4b18-9dd5-bc687894...,False,6803620337242231,False,...,false,witsml-server-reference,KongsbergDigital,Block/Topdrive Position Compensated,2022-10-12T08:05:02.000000Z,2022-02-06T01:26:18.000000Z,BLOCKCOMP,BLOCKCOMP,double,BLOCKCOMP
9,6043984512883,1da9e641-852b-4d02-9d68-89fe127384d5,RH_YP_COR50,Surface Logging Data - Time RT,a2bbffd2-55d5-4e8f-9183-ad8b83399300,NO 16/2-H-1 HT2,dw:witsml:log.1da9e641-852b-4d02-9d68-89fe1273...,False,8683220645738181,False,...,false,witsml-server-reference

In [ ]:

#Get name and id of wellbore
pd_wellbores = _pd_witsml_flattened[['nameWellbore', 'uidWellbore']]
#Get rid of duplicates
pd_wellbores = pd_wellbores.drop_duplicates(subset=['uidWellbore'])
#Create dictionary 
wellbore_map = {row['nameWellbore']: row['uidWellbore'] for index, row in pd_wellbores.iterrows()}

#Get name and id of sensor
pd_sensors = _pd_witsml_flattened[['name', 'id', 'external_id']]
#Get rid of duplicates
pd_sensors = pd_sensors.drop_duplicates(subset=['id'])
#Create dictionary 

sensor_map = {row['external_id']: row['id'] for index, row in pd_sensors.iterrows()}
sensor_map



In [163]:




class CustomDashboard(param.Parameterized):
    
    wellbore = param.ObjectSelector(default=pd_wellbores['nameWellbore'].iloc[0], objects=list(pd_wellbores['nameWellbore']))
    sensor = param.ObjectSelector(default=None, objects=[])

    
    @param.depends('wellbore', watch=True)
    def update_sensors(self):
        wellbore_id = wellbore_map[self.wellbore]

        filtered_sensors = _pd_witsml_flattened[_pd_witsml_flattened['uidWellbore'] == wellbore_id]
        sensor_names = list(filtered_sensors['sensorName'])
        self.param['sensor'].objects = sensor_names
        if sensor_names:
                self.sensor = sensor_names[0]
        else:
            self.sensor= None
        
        
        

    @param.depends('wellbore', 'sensor')
    def get_accessible_value(self):
        wellbore_id= wellbore_map[self.wellbore]
        sensor_data = _pd_witsml_flattened[(_pd_witsml_flattened['uidWellbore'] == wellbore_id) & (_pd_witsml_flattened['sensorName'] == self.sensor)]
        if not sensor_data.empty:
            return sensor_data['id'].iloc[0]
        return None
    
    @param.depends('wellbore', 'sensor')
    def view(self):
        accessible_value = self.get_accessible_value()
        if accessible_value:
                return pn.pane.Str(f'Selected Sensor ID: {accessible_value}')
        return pn.pane.Str('No sensor selected')
    
    

    @param.depends('sensor')
    def plot(self):
        if self.sensor:
            sensor_id = int(self.get_accessible_value())
            sensor_data = client.time_series.data.retrieve(id=sensor_id,start=MIN_TIMESTAMP_MS,  end=MAX_TIMESTAMP_MS + 1)
            pd_sensor_data = sensor_data.to_pandas()
            
            pd_sensor_data = pd_sensor_data.drop(pd_sensor_data.columns[[0]], axis=1)  # _pd_witsml_flattened.columns is zero-based pd.Index
            pd_sensor_data['Date'] = pd_sensor_data.index
            



            if not pd_sensor_data.empty:
                plot = pd_sensor_data.hvplot.line(x='Value', y='Date', title=f'Sensor Data for {self.sensor}')
                return plot
        return pn.pane.Str('No data available for the selected sensor')
    
    
    def panel(self):
        return pn.Column(
            pn.Param(self.param, widgets={'wellbore': pn.widgets.Select, 'sensor': pn.widgets.Select}),
            self.view, self.plot
        )
    


# Instantiate the dashboard
dashboard = CustomDashboard()

# Trigger initial sensor update
dashboard.update_sensors()


# Display the dashboard
dashboard.panel().show()





Launching server at http://localhost:62010
